In [173]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
import statsmodels.api as sm
from sklearn.impute import SimpleImputer
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [174]:


# Load the CSV file into a DataFrame
#df = pd.read_csv(r"C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\filtered_eras_dates_combined.csv")
df = pd.read_csv(r"C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\final_combined_data.csv", low_memory=False)


In [175]:
df.columns

Index(['day_of_week', 'date', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_iata', 'origin_city', 'dest_iata',
       'dest_city', 'crs_dep_time', 'dep_time', 'taxi_out', 'wheels_off',
       'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time', 'cancelled',
       'diverted', 'crs_elapsed_time', 'actual_elapsed_time', 'air_time',
       'flights', 'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'origin_state', 'dest_state',
       'origin_latitude', 'origin_longitude', 'dest_latitude',
       'dest_longitude', 'type_of_airport', 'elevation_ft', 'municipality',
       'scheduled_service', 'unique_id', 'id', 'length_ft', 'width_ft',
       'surface', 'lighted', 'closed', 'le_ident', 'le_displaced_threshold_ft',
       'he_ident', 'he_displaced_threshold_ft', 'latitude_orig',
       'longitude_orig', 'elevation_orig', 'prcp_orig', 'snow_orig',
       'snwd_orig', 'tmax_orig', 'tmin_orig', 'tobs', 'iata_dest_d

In [176]:
# Check the shape and data types
print(df.shape)
print(df.dtypes)

# Display summary statistics
print(df.describe(include='all'))

# Check the data type of the column
print("Data type of 'eras_tour':", df['eras_tour'].dtype)

# Print unique values to see what they look like
print("Unique values in 'eras_tour':", df['eras_tour'].unique())


(15118733, 78)
day_of_week           object
date                  object
op_unique_carrier     object
tail_num              object
op_carrier_fl_num      int64
                      ...   
prcp_dest            float64
snow_dest            float64
snwd_dest            float64
tmax_dest            float64
tmin_dest            float64
Length: 78, dtype: object
       day_of_week        date op_unique_carrier  tail_num  op_carrier_fl_num  \
count     15118733    15118733          15118733  15118733       1.511873e+07   
unique           7         366                15      5956                NaN   
top         monday  2023-11-26                aa    n233nn                NaN   
freq       2263537       45330           2858240     50177                NaN   
mean           NaN         NaN               NaN       NaN       2.330569e+03   
std            NaN         NaN               NaN       NaN       1.531463e+03   
min            NaN         NaN               NaN       NaN       1.000000

In [177]:
#Check for missing values and their distribution:
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])


latitude_orig     12865978
longitude_orig    12865978
elevation_orig    12869368
prcp_orig         12879327
snow_orig         13163472
snwd_orig         13477068
tmax_orig         12916182
tmin_orig         12916577
tobs              15102106
iata_dest_dest    12865978
eras_tour         15086378
prcp_dest         12879327
snow_dest         13163472
snwd_dest         13477068
tmax_dest         12916182
tmin_dest         12916577
dtype: int64


In [178]:
# Check if the 'date' column is already in datetime format
if df['date'].dtype == 'object':
    # Convert 'date' to datetime format
    df['date'] = pd.to_datetime(df['date'])

In [220]:
# Filter rows where 'eras_tour' is 'tour_date'
tour_dates = df[df['eras_tour'] == 'tour_date']


In [260]:
df['eras_tour'].head()

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: eras_tour, dtype: object

In [261]:
# Get the unique dates for these rows
unique_tour_dates = tour_dates['date'].unique()

In [262]:
print(tour_dates.head())

# Print the unique tour dates
print(unique_tour_dates)

       day_of_week       date op_unique_carrier tail_num  op_carrier_fl_num  \
163011      friday 2023-05-05                ua   n11206                394   
163012      friday 2023-05-05                ua   n11206                394   
163020      friday 2023-05-05                ua   n11206                394   
163028      friday 2023-05-05                ua   n11206                394   
163042      friday 2023-05-05                ua   n12225                740   

       origin_iata origin_city dest_iata dest_city  crs_dep_time  ...  \
163011         bna   nashville       ewr    newark          1079  ...   
163012         bna   nashville       ewr    newark          1079  ...   
163020         bna   nashville       ewr    newark          1079  ...   
163028         bna   nashville       ewr    newark          1079  ...   
163042         bna   nashville       iah   houston           380  ...   

        nfl_game  delay_columns  latitude_dest  longitude_dest  \
163011        no    

In [300]:

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter rows where 'eras_tour' is 'tour_date'
tour_dates = df[df['eras_tour'] == 'tour_date']['date']

# Create a list to hold the date ranges
date_ranges = []

# Loop through each tour date to create the date ranges
for tour_date in tour_dates:
    # Create a date range for ±3 days around each tour date
    start_date = tour_date - pd.DateOffset(days=3)
    end_date = tour_date + pd.DateOffset(days=3)
    date_ranges.append((start_date, end_date))

# Print the date ranges
for start, end in date_ranges:
    print(f"Start: {start.date()}, End: {end.date()}")


Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, End: 2023-05-08
Start: 2023-05-02, E

In [302]:
df.head()

,day_of_week,date,op_unique_carrier,tail_num,op_carrier_fl_num,origin_iata,origin_city,dest_iata,dest_city,crs_dep_time,...,nfl_game,delay_columns,latitude_dest,longitude_dest,elevation_dest,prcp_dest,snow_dest,snwd_dest,tmax_dest,tmin_dest
0,monday,2023-05-01,9e,n131ev,4888,cvg,cincinnati,msp,minneapolis,754,...,no,0.0,44.8820,-93.2218,896.0,NaN,NaN,NaN,NaN,NaN
1,monday,2023-05-01,oo,n905ev,5331,sbn,south bend,ord,chicago,460,...,no,0.0,41.9786,-87.9047,799.0,NaN,NaN,NaN,NaN,NaN
2,monday,2023-05-01,oo,n908ev,4761,psp,palm springs,lax,los angeles,524,...,no,0.0,33.9425,-118.4080,477.0,NaN,NaN,NaN,NaN,NaN
3,monday,2023-05-01,oo,n908ev,4761,psp,palm springs,lax,los angeles,524,...,no,0.0,33.9425,-118.4080,477.0,NaN,NaN,NaN,NaN,NaN
4,monday,2023-05-01,oo,n908ev,5529,lax,los angeles,rdd,redding,660,...,no,0.0,40.5090,-122.2930,125.0,NaN,NaN,NaN,NaN,NaN


In [304]:
print(df[['arr_time', 'crs_arr_time']])

          arr_time  crs_arr_time
0            790.0           821
1            461.0           464
2            589.0           576
3            589.0           576
4            771.0           763
...            ...           ...
15118728     773.0           771
15118729    1289.0          1287
15118730    1226.0          1220
15118731    1266.0          1257
15118732     840.0           849

[15118733 rows x 2 columns]


In [305]:
# Convert arr_time and crs_arr_time columns to datetime (if they are not already)
df['arr_time'] = pd.to_datetime(df['arr_time'], errors='coerce')
df['crs_arr_time'] = pd.to_datetime(df['crs_arr_time'], errors='coerce')

# Extract the time in HHMM format for both arr_time and crs_arr_time columns
df['arr_time_HHMM'] = df['arr_time'].dt.strftime('%H%M')
df['crs_arr_time_HHMM'] = df['crs_arr_time'].dt.strftime('%H%M')

# Verify the result
print(df[['arr_time_HHMM', 'crs_arr_time_HHMM']].head())


  arr_time_HHMM crs_arr_time_HHMM
0          0000              0000
1          0000              0000
2          0000              0000
3          0000              0000
4          0000              0000


In [306]:
df.columns

Index(['day_of_week', 'date', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_iata', 'origin_city', 'dest_iata',
       'dest_city', 'crs_dep_time', 'dep_time', 'taxi_out', 'wheels_off',
       'wheels_on', 'taxi_in', 'crs_arr_time', 'arr_time', 'cancelled',
       'diverted', 'crs_elapsed_time', 'actual_elapsed_time', 'air_time',
       'flights', 'distance', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'origin_state', 'dest_state',
       'origin_latitude', 'origin_longitude', 'dest_latitude',
       'dest_longitude', 'type_of_airport', 'elevation_ft', 'municipality',
       'scheduled_service', 'unique_id', 'id', 'length_ft', 'width_ft',
       'surface', 'lighted', 'closed', 'le_ident', 'le_displaced_threshold_ft',
       'he_ident', 'he_displaced_threshold_ft', 'latitude_orig',
       'longitude_orig', 'elevation_orig', 'prcp_orig', 'snow_orig',
       'snwd_orig', 'tmax_orig', 'tmin_orig', 'tobs', 'iata_dest_d

In [ ]:
# Prepare the data
# Create a new column to indicate if a flight is within the ±3 days of tour dates
df['is_tour_window'] = df['date'].apply(
    lambda x: any(start <= x <= end for start, end in date_ranges)
)

#Convert boolean to int (1 for True, 0 for False)
df['is_tour_window'] = df['is_tour_window'].astype(int)

# Calculate correlation coefficients
# Calculate the Pearson correlation coefficient
pearson_corr, pearson_p_value = pearsonr(df['is_tour_window'], df['delay_in_min'])

# Calculate the Spearman correlation coefficient
spearman_corr, spearman_p_value = spearmanr(df['is_tour_window'], df['delay_in_min'])

#  Display the results
print(f"Pearson Correlation: {pearson_corr}, p-value: {pearson_p_value}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {spearman_p_value}")

In [ ]:

df['delay_in_min'].hist(bins=1000)
plt.title('Total Delay Time Distribution (Minutes)')
plt.xlabel('Total Delay Time (Minutes)' )
plt.ylabel('Frequency')

# Set x-axis limits to zoom in (adjust values as needed)
plt.xlim(-.25, .25)  # Change these values based on data distribution

plt.show()


In [ ]:
df.columns

In [ ]:
#Box plots to identify outliers:
plt.figure(figsize=(12, 6))  # Adjust the size as needed
sns.boxplot(x='day_of_week', y='delay_in_min', data=df)
plt.title('Total Delay Time by Day of Week')
plt.xticks(rotation=45)
plt.show()



In [ ]:
# Compare tour date metrics to non-tour date metrics

Average Delay Comparison:

Tour Dates: The average delay for flights around the tour dates (±3 days) is 80.46 minutes, which is significantly higher than the average delay for non-tour flights.
Non-Tour Dates: The average delay for flights on non-tour dates is 63.34 minutes.
This suggests that flights around tour dates tend to experience longer delays on average compared to non-tour flights. This could be due to a variety of factors such as higher traffic, operational constraints, or specific scheduling issues around tour dates.

Median Delay Comparison:

The median delay for both tour and non-tour flights is relatively close: 6 minutes for tour dates and 7 minutes for non-tour dates. This suggests that the central tendency (or the "typical" delay) is similar across both types of flights, but the average delay is higher for tour dates because of some larger delays (likely outliers).

In [ ]:
df['delay_in_min'].mean()

In [ ]:
df['delay_in_min'].median()

In [ ]:
# 'date' to datetime format
df['date'] = pd.to_datetime(df['date'])

# create additional features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day


In [ ]:

# Group by 'op_carrier_fl_num' and 'day_of_week' to calculate the total delay and count of flights
delay_and_count =  df.groupby(['op_carrier_fl_num', 'day_of_week'])['delay_in_min'].agg(['sum', 'count']).reset_index()


In [ ]:

# Calculate the average delay per flight by dividing the sum of delays by the count of flights
delay_and_count['average_delay_per_flight'] = delay_and_count['sum'] / delay_and_count['count']

# Step 3: Merge the calculated average delay back to the original DataFrame
df = pd.merge(df, delay_and_count[['op_carrier_fl_num', 'day_of_week', 'average_delay_per_flight']], 
                          on=['op_carrier_fl_num', 'day_of_week'], how='left')

# Check the result
print(df[['date', 'op_carrier_fl_num', 'day_of_week', 'delay_in_min', 'average_delay_per_flight']].head())


In [ ]:

df['average_delay_per_flight'].hist(bins=1000)
plt.title('average_delay_per_flight   Distribution (Minutes)')
plt.xlabel('average_delay_per_flight   Time (Minutes)' )
plt.ylabel('Frequency')

# Set x-axis limits to zoom in (adjust values as needed)
plt.xlim(0, 10)  # Change these values based on data distribution
plt.ylim(0,25000)
plt.show()

In [ ]:
df['delay_in_min'].head()

In [ ]:
# Group by 'day_of_week' and calculate the average delay per day of the week
avg_delay_per_day = df.groupby('day_of_week')['average_delay_per_flight'].mean()

# Plot the results
# Create a bar plot
plt.figure(figsize=(10, 6))
avg_delay_per_day.plot(kind='bar', color='skyblue')

plt.title('Average Delay per Flight by Day of the Week', fontsize=16)
plt.xlabel('Day of the Week', fontsize=12)
plt.ylabel('Average Delay (minutes)', fontsize=12)
plt.xticks(ticks=range(7), labels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Columns:", categorical_cols)


In [ ]:
# covert to category data type
for col in categorical_cols:
    df[col] = df[col].astype('category')

In [ ]:
# Create a dictionary to store the counts of each categorical feature
feature_counts = {col: df[col].value_counts().sum() for col in categorical_cols}

# Sort the features by count and get the top 15
top_features = sorted(feature_counts, key=feature_counts.get, reverse=True)[:15]

# Plot the distribution of the top 15 most frequent features
for col in top_features:
    plt.figure(figsize=(10, 6))
    sns.countplot(y=col, data=df)
    plt.title(f'Distribution of {col}')
    plt.show()


In [ ]:
# Magnitude Standardization

scaler = StandardScaler()

# Select numerical columns for scaling
numerical_cols = df.select_dtypes(include=['float64', 'int']).columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [ ]:
# Prepare the data
# Create a new column to indicate if a flight is within the ±3 days of tour dates
df['is_tour_window'] = df['date'].apply(
    lambda x: any(start <= x <= end for start, end in date_ranges)
)

# Step 2: Convert boolean to int (1 for True, 0 for False)
df['is_tour_window'] = df['is_tour_window'].astype(int)

# Step 3: Calculate correlation coefficients
# Calculate the Pearson correlation coefficient
pearson_corr, pearson_p_value = pearsonr(df['is_tour_window'], df['delay_in_min'])

# Calculate the Spearman correlation coefficient
spearman_corr, spearman_p_value = spearmanr(df['is_tour_window'], df['delay_in_min'])

# Step 4: Display the results
print(f"Pearson Correlation: {pearson_corr}, p-value: {pearson_p_value}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {spearman_p_value}")

Both correlation coefficients are very close to zero, indicating a negligible relationship between being within the ±3-day window of tour dates and the adjusted elapsed time for delays. The p-values are above the typical significance threshold of 0.05, suggesting that the correlations observed are not statistically significant. In other words, we do not have enough evidence to conclude that there's a meaningful correlation between tour dates and delays.

In [ ]:

#Create More Relevant Features

# identify how specific events (like tour dates and NFL game days)
# and delays affect flight delays.

df['is_tour_date'] = df['is_tour_date'].apply(lambda x: 1 if x == '1' else 0)


In [ ]:
df.columns

In [ ]:
print(df['is_nfl_game_day'].unique())



In [ ]:
print(df['is_nfl_game_day'].head())

In [ ]:
# Prepare the data
# Create a new column to indicate if a flight is within the ±3 days of tour dates
df['is_tour_window'] = df['date'].apply(
    lambda x: any(start <= x <= end for start, end in date_ranges)
)

# Step 2: Convert boolean to int (1 for True, 0 for False)
df['is_tour_window'] = df['is_tour_window'].astype(int)

# Step 3: Calculate correlation coefficients
# Calculate the Pearson correlation coefficient
pearson_corr, pearson_p_value = pearsonr(df['is_tour_window'], df['delay_in_min'])

# Calculate the Spearman correlation coefficient
spearman_corr, spearman_p_value = spearmanr(df['is_tour_window'], df['delay_in_min'])

# Step 4: Display the results
print(f"Pearson Correlation: {pearson_corr}, p-value: {pearson_p_value}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {spearman_p_value}")

In [ ]:
# Split the data into training and test sets
X_train_sample, X_test_sample, y_train_sample, y_test_sample = train_test_split(
    X_sample, y_sample, test_size=0.2, random_state=42)

In [ ]:
# Define features (X) and target (y)
df['has_delay'] = df[['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].notna().any(axis=1).astype(int)

# Split the data into features (X) and target (y)
X_sample = df[['day_of_week', 'crs_dep_time', 'is_tour_date', 'has_delay', 'is_nfl_game_day']]
y_sample = df['delay_in_min']

In [ ]:
y_train_sample = y_train_sample.map({'True': 1, 'False': 0})
y_test_sample = y_test_sample.map({'True': 1, 'False': 0})


In [ ]:
print(y_train_sample.head())
print(y_test_sample.head())


In [ ]:
y_train_sample = y_train_sample.map({'True': 1, 'False': 0})
y_test_sample = y_test_sample.map({'True': 1, 'False': 0})

y_train_sample = pd.to_numeric(y_train_sample, errors='coerce')
y_test_sample = pd.to_numeric(y_test_sample, errors='coerce')

print(y_train_sample.dtype)
print(y_test_sample.dtype)


In [ ]:
# Check for any NaN values
print(y_train_sample.isnull().sum())
print(y_test_sample.isnull().sum())

# Check for any non-numeric values (if any)
print(y_train_sample[~y_train_sample.apply(lambda x: isinstance(x, (int, float)))])
print(y_test_sample[~y_test_sample.apply(lambda x: isinstance(x, (int, float)))])


In [ ]:
df.columns

Low R-squared: The model's R-squared value of 0.009 is very low, which means that the independent variables you've chosen do not explain much of the variance in delay_in_min. You might need to consider adding more relevant features or revising your existing features to improve model performance.

Multicollinearity Concerns: The warning about the smallest eigenvalue (1.86e-30) could indicate multicollinearity problems. This can happen if some of your independent variables are highly correlated with each other, which can make the coefficient estimates unstable. You can check for multicollinearity using variance inflation factors (VIF) to identify problematic predictors.

is_tour_window variable: The very small coefficient and high p-value suggest that this variable does not contribute much to explaining the variance in delay_in_min

In [ ]:
print(X.dtypes)
print(y.dtypes)

In [ ]:
print(X.isnull().sum())
print(y.isnull().sum())

# regression model performance
Mean Squared Error (MSE): A lower MSE indicates better performance. In this case, the Random Forest model has the lowest MSE, suggesting it's the best at predicting the target variable on this sample.

R² Score: The R² value indicates how well the model explains the variance in the target variable, Values closer to 1 indicate a good fit, while values less than 0 suggest that the model performs worse than a horizontal line (mean of the target variable).
Here, the Random Forest model also has the highest R² score, but all models are relatively low, indicating that there may be issues with the features or the data itself (like noise or irrelevance).

## Visualization of Delay Distributions for Tour Dates vs Non-Tour Dates

## Bar Charts for Average and Median Delays 

In [ ]:
# Data for average and median delays
summary_data = {
    'Type': ['Tour Dates', 'Non-Tour Dates'],
    'Average Delay': [85.23, 63.44],
    'Median Delay': [5.0, 9.0]
}

summary_df = pd.DataFrame(summary_data)

# Bar Chart
summary_df.set_index('Type').plot(kind='bar', figsize=(10, 6))
plt.title('Average and Median Delay Comparison')
plt.ylabel('Delay (minutes)')
plt.xticks(rotation=0)
plt.legend(title='Metric')
plt.show()


In [ ]:
# Prepare the data
# Create a new column to indicate if a flight is within the ±3 days of tour dates
df['is_tour_window'] = df['date'].apply(
    lambda x: any(start <= x <= end for start, end in date_ranges)
)

# Step 2: Convert boolean to int (1 for True, 0 for False)
df['is_tour_window'] = df['is_tour_window'].astype(int)

# Step 3: Calculate correlation coefficients
# Calculate the Pearson correlation coefficient
pearson_corr, pearson_p_value = pearsonr(df['is_tour_window'], df['delay_in_min'])

# Calculate the Spearman correlation coefficient
spearman_corr, spearman_p_value = spearmanr(df['is_tour_window'], df['delay_in_min'])

# Step 4: Display the results
print(f"Pearson Correlation: {pearson_corr}, p-value: {pearson_p_value}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {spearman_p_value}")

Coefficients Interpretation:
is_tour_window: Coefficient = 2.7664 (p-value = 0.011)

The positive coefficient suggests that being within the tour window is associated with an increase in adjusted elapsed time by about 2.77 minutes on average, which is statistically significant.
weather_delay: Coefficient = -0.0907 (p-value = 0.022)

This negative coefficient implies that for each additional minute of weather delay, the adjusted elapsed time decreases by about 0.09 minutes. This might seem counterintuitive but could indicate how weather-related delays are factored in or reported.
Airline Coefficients: Various airlines show different effects on the adjusted elapsed time:

For example, op_unique_carrier_aa has a coefficient of 41.44, meaning flights from American Airlines tend to have longer adjusted elapsed times compared to the baseline carrier.

In [ ]:

# Group delays by airline carrier
grouped_delays = [group['delay_in_min'].values for name, group in df.groupby('op_unique_carrier')]

# Perform one-way ANOVA
anova_result = f_oneway(*grouped_delays)

# Print the results
print(f"ANOVA F-statistic: {anova_result.statistic}, p-value: {anova_result.pvalue}")


In [ ]:
# Calculate residuals
residuals = model.resid

# Residual plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=model.fittedvalues, y=residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

